In [10]:
%pip install -q pandas numpy yfinance tabulate rich



Note: you may need to restart the kernel to use updated packages.


In [11]:
import math
from dataclasses import dataclass
from typing import Dict, List

import numpy as np
import pandas as pd
import yfinance as yf
from tabulate import tabulate

In [12]:
FTSE100_TICKERS = [
    "III.L","ADM.L","AAF.L","ALW.L","AAL.L","ANTO.L","AHT.L","ABF.L","AZN.L","AUTO.L","AV.L","BAB.L","BA.L","BARC.L","BTRW.L","BEZ.L","BKG.L",
    "BP.L","BATS.L","BT-A.L","BNZL.L","CNA.L","CCEP.L","CCH.L","CPG.L","CTEC.L","CRDA.L","DCC.L","DGE.L","DPLM.L","EDV.L","ENT.L","EZJ.L","EXPN.L",
    "FCIT.L","FRES.L","GAW.L","GLEN.L","GSK.L","HLN.L","HLMA.L","HIK.L","HSX.L","HWDN.L","HSBA.L","ICG.L","IHG.L","IMI.L","IMB.L","INF.L","IAG.L",
    "ITRK.L","JD.L","KGF.L","LAND.L","LGEN.L","LLOY.L","LMP.L","LSEG.L","MNG.L","MKS.L","MRO.L","MNDI.L","NG.L","NWG.L","NXT.L","PSON.L","PSH.L",
    "PSN.L","PHNX.L","PCT.L","PRU.L","RKT.L","REL.L","RTO.L","RMV.L","RIO.L","RR.L","SGE.L","SBRY.L","SDR.L","SMT.L","SGRO.L","SVT.L","SHEL.L",
    "SMIN.L","SN.L","SPX.L","SSE.L","STAN.L","STJ.L","TW.L","TSCO.L","ULVR.L","UU.L","UTG.L","VOD.L","WEIR.L","WTB.L","WPP.L"
]
len(FTSE100_TICKERS)

100

In [13]:
@dataclass
class Weights:
    momentum: float = 0.4
    value: float = 0.2
    quality: float = 0.2
    low_volatility: float = 0.2
    
def annualised_volatility(prices: pd.Series) -> float:
    returns = prices.pct_change().dropna()
    if len(returns) < 60:
        return np.nan
    daily_vol = returns.std()
    return float(daily_vol * math.sqrt(252))

def trailing_return(prices, window: pd.Series) -> float:
    if len(prices) < window:
        return np.nan
    return float((prices.iloc[-1]/prices.iloc[-window] - 1))

def zscore(series: pd.Series) -> pd.Series:
    return (series - series.mean()) / series.std(ddof=0)
        

In [14]:
lookback = "5y"
rows: List[Dict] = []
for ticker in FTSE100_TICKERS:
    data = yf.download(
        ticker,
        period = lookback,
        interval = "1d",
        auto_adjust = True,
        group_by = "column",
        progress = False
    )
    if data.empty:
        continue
    prices = data["Close"].dropna()
    
    info = (yf.Ticker(ticker).info or {})
    
    rows.append({
        "Stock": ticker,
        "Name": info.get("shortName"),
        "Sector": info.get("sector"),
        "Trailing PE": info.get("trailingPE"),
        "Market Cap": info.get("marketCap"),
        "Return 3m": trailing_return(prices,63),
        "Return 6m": trailing_return(prices,126),
        "Return 12m": trailing_return(prices,252),
        "Annual Vol": annualised_volatility(prices),
    })
    
raw_data = pd.DataFrame(rows)
raw_data.head()

,Stock,Name,Sector,Trailing PE,Market Cap,Return 3m,Return 6m,Return 12m,Annual Vol
0,III.L,"""3i Group Plc""",None,7.439539,37958598656,-0.049098,-0.012607,0.273782,0.263448
1,ADM.L,ADMIRAL GROUP PLC ORD 0.1P,Financial Services,13.169235,10772137984,0.067352,0.239368,0.222668,0.263300
2,AAF.L,AIRTEL AFRICA PLC ORD USD0.50,Communication Services,30.942860,7898600448,0.192270,0.533160,0.969813,0.365013
3,ALW.L,"""Alliance Witan PLC""",None,34.711110,4916551168,0.046881,-0.010945,0.055882,0.159351
4,AAL.L,ANGLO AMERICAN PLC ORD USD0.549,Basic Materials,NaN,24097024000,-0.022859,-0.007021,0.089319,0.400378


In [15]:
w = Weights()
df = raw_data.copy()

df["Mom"] = (
    0.5 * zscore(df["Return 12m"]).fillna(0) +
    0.3 * zscore(df["Return 6m"]).fillna(0) +
    0.2 * zscore(df["Return 3m"]).fillna(0)
)
df["Val"] = (-zscore(df["Trailing PE"]).fillna(0))
df["Qual"] = zscore(df["Market Cap"]).fillna(0)
df["LowVol"] = (-zscore(df["Annual Vol"]).fillna(0))
df["Score"] = (
    w.momentum * df["Mom"] + w.value * df["Val"] + w.quality * df["Qual"] + w.low_volatility * df["LowVol"]
)

ranked = df.sort_values("Score", ascending = False).reset_index(drop = True)
ranked.head(10)
                
              
                      

,Stock,Name,Sector,Trailing PE,Market Cap,Return 3m,Return 6m,Return 12m,Annual Vol,Mom,Val,Qual,LowVol,Score
0,FRES.L,FRESNILLO PLC ORD USD0.50,Basic Materials,40.652176,13779917824,0.411493,1.398165,2.678601,0.397657,5.547325,-0.760024,-0.296723,-1.688970,1.669787
1,HSBA.L,HSBC HOLDINGS PLC ORD $0.50 (UK,Financial Services,12.728000,165510447104,0.093084,0.042305,0.450080,0.262070,0.460517,0.577578,4.208194,0.389617,1.219285
2,AZN.L,ASTRAZENECA PLC ORD SHS $0.25,Healthcare,30.417303,185315688448,0.117274,-0.009049,-0.083509,0.238104,-0.221529,-0.269761,4.796216,0.757019,0.968083
3,SHEL.L,SHELL PLC ORD EUR0.07,Energy,16.569277,160312049664,0.109871,0.071518,0.065725,0.270235,0.059833,0.393576,4.053852,0.264441,0.966307
4,BATS.L,BRITISH AMERICAN TOBACCO PLC OR,Consumer Defensive,29.402878,89852289024,0.168916,0.295462,0.409907,0.214146,0.881082,-0.221169,1.961884,1.124292,0.925434
5,ULVR.L,UNILEVER PLC ORD 3 1/9P,Consumer Defensive,24.386599,115996549120,0.019699,0.024867,-0.046233,0.186905,-0.282539,0.019117,2.738114,1.541904,0.746812
6,BARC.L,BARCLAYS PLC ORD 25P,Financial Services,8.836585,50851336192,0.107873,0.239158,0.620001,0.330064,0.962811,0.763981,0.803936,-0.652756,0.568157
7,RR.L,ROLLS-ROYCE HOLDINGS PLC ORD SH,Industrials,15.838236,89675431936,0.231224,0.372246,1.244922,0.541523,2.100805,0.428594,1.956633,-3.894460,0.538476
8,IMB.L,IMPERIAL BRANDS PLC ORD 10P,Consumer Defensive,10.279264,25025851392,0.059597,0.088644,0.379796,0.210972,0.386551,0.694875,0.037171,1.172950,0.535620
9,STAN.L,STANDARD CHARTERED PLC ORD USD0,Financial Services,10.202592,31946805248,0.195205,0.109785,0.837065,0.340851,1.184985,0.698548,0.242656,-0.818124,0.498610


In [16]:
from pathlib import Path

base = Path.home() / "Documents" / "Python" / "FTSE 100"

outdir = base / "Data Outputs"
outdir.mkdir(exist_ok = True)

my_columns = ["Stock", "Name", "Sector", "Return 3m", "Return 6m", "Return 12m",
    "Annual Vol", "Trailing PE", "Score"]

ranked.to_csv(outdir / "FTSE 100 Ranking.csv")
print("Saved.")

Saved.
